In [2]:
import os
import subprocess
import glob
import tqdm
from multiprocessing import Pool, cpu_count

# 配置参数
VIDEO_DIR = "../../LongVALE/raw_videos_train/video_train_7240/"
AUDIO_DIR = "../../LongVALE/raw_videos_train/video_train_7240/"
SAMPLE_RATE = 16000
# 并行进程数（建议设为CPU核心数的70%-80%，避免IO过载）
NUM_PROCESSES = max(1, cpu_count() - 72)
# 是否清理已存在的WAV文件（True=清理，False=不清理）
CLEAN_EXISTING_WAVS = True

def clean_existing_wavs(audio_dir):
    """
    清理指定目录下所有的WAV文件（针对损坏的旧文件）
    """
    if not CLEAN_EXISTING_WAVS:
        print("跳过清理WAV文件（CLEAN_EXISTING_WAVS=False）")
        return
    
    # 获取目录下所有WAV文件
    wav_paths = glob.glob(os.path.join(audio_dir, "*.wav"))
    if not wav_paths:
        print("未找到需要清理的WAV文件")
        return
    
    # 批量删除WAV文件（带进度条）
    print(f"发现 {len(wav_paths)} 个WAV文件，开始清理...")
    for wav_path in tqdm.tqdm(wav_paths, desc="清理损坏WAV文件"):
        try:
            os.remove(wav_path)
        except Exception as e:
            print(f"删除失败: {wav_path} | 错误: {str(e)[:50]}")
    print(f"清理完成，共删除 {len(wav_paths)} 个WAV文件（失败的已提示）")

def extract_audio(video_path):
    """
    单个视频的音频提取函数（供多进程调用）
    """
    try:
        # 生成音频文件名（mp4→wav）
        filename = os.path.basename(video_path)
        audio_path = os.path.join(AUDIO_DIR, filename.replace(".mp4", ".wav"))
        
        # 跳过已生成的音频文件（避免重复处理，清理后这一步实际不会触发）
        if os.path.exists(audio_path) and os.path.getsize(audio_path) > 0:
            return f"跳过: {filename}"
        
        # 构造ffmpeg命令（直接提取音频，效率远高于librosa）
        # -vn: 禁用视频流 | -ac 1: 单声道 | -ar: 采样率 | -y: 覆盖已存在文件
        cmd = [
            "ffmpeg",
            "-i", video_path,       # 输入视频
            "-vn",                  # 只处理音频，忽略视频
            "-ac", "1",             # 单声道
            "-ar", str(SAMPLE_RATE),# 采样率
            "-acodec", "pcm_s16le", # WAV编码格式（和sf.write一致）
            "-y",                   # 覆盖已存在的音频文件
            audio_path              # 输出音频路径
        ]
        
        # 执行ffmpeg命令（静默执行，不输出冗余日志）
        subprocess.run(
            cmd,
            stdout=subprocess.DEVNULL,  # 屏蔽标准输出
            stderr=subprocess.DEVNULL,  # 屏蔽标准错误
            check=True                  # 执行失败抛出异常
        )
        return f"成功: {filename}"
    except Exception as e:
        return f"失败: {filename} | 错误: {str(e)[:50]}"  # 截断错误信息，避免过长

if __name__ == "__main__":
    # 第一步：清理损坏的旧WAV文件
    clean_existing_wavs(AUDIO_DIR)
    
    # 第二步：获取所有mp4视频路径
    video_paths = glob.glob(os.path.join(VIDEO_DIR, "*.mp4"))
    print(f"\n待处理视频数: {len(video_paths)}")
    print(f"并行进程数: {NUM_PROCESSES}")
    
    # 第三步：多进程批量提取音频（带进度条）
    with Pool(NUM_PROCESSES) as pool:
        results = list(tqdm.tqdm(
            pool.imap(extract_audio, video_paths),
            total=len(video_paths),
            desc="提取音频进度"
        ))
    
    # 第四步：打印处理结果统计
    success = [r for r in results if r.startswith("成功")]
    skipped = [r for r in results if r.startswith("跳过")]
    failed = [r for r in results if r.startswith("失败")]
    print(f"\n处理完成：")
    print(f"成功: {len(success)} | 跳过: {len(skipped)} | 失败: {len(failed)}")
    if failed:
        print("失败列表：")
        for f in failed:
            print(f)

发现 2595 个WAV文件，开始清理...


清理损坏WAV文件:   0%|          | 0/2595 [00:00<?, ?it/s]

清理损坏WAV文件: 100%|██████████| 2595/2595 [00:19<00:00, 131.84it/s]


清理完成，共删除 2595 个WAV文件（失败的已提示）

待处理视频数: 7240
并行进程数: 40


提取音频进度: 100%|██████████| 7240/7240 [33:36<00:00,  3.59it/s]  


处理完成：
成功: 7240 | 跳过: 0 | 失败: 0
